In [1]:
!pip install pandera --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.6/259.6 kB 5.3 MB/s eta 0:00:00


In [2]:
!pip install ipykernel==5.5.6 notebook==6.5.5

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [3]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import recommenders
print(recommenders.__version__)

1.2.0


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
Pandas version: 2.1.4
Tensorflow version: 2.17.0


In [7]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

In [8]:
import pandas as pd

df_D = pd.read_csv("tn_visit_area_info_travel_D.csv")

# df.columns
df_D = df_D[['TRAVEL_ID', 'VISIT_AREA_ID', 'REVISIT_INTENTION', 'VISIT_START_YMD']]
df_D = df_D.dropna()
df_D.rename(columns={
    'TRAVEL_ID': 'userID',
    'VISIT_AREA_ID': 'itemID',
    'REVISIT_INTENTION': 'rating',
    'VISIT_START_YMD': 'timestamp'
}, inplace=True)
df_D.shape

(11266, 4)

In [9]:
import pandas as pd

df_E = pd.read_csv("tn_visit_area_info_travel.csv")

# df.columns
df_E = df_E[['TRAVEL_ID', 'VISIT_AREA_ID', 'REVISIT_INTENTION', 'VISIT_START_YMD']]
df_E = df_E.dropna()
df_E.rename(columns={
    'TRAVEL_ID': 'userID',
    'VISIT_AREA_ID': 'itemID',
    'REVISIT_INTENTION': 'rating',
    'VISIT_START_YMD': 'timestamp'
}, inplace=True)
# columns = ['userID', 'itemID', 'rating', 'timestamp']
# df.columns = columns
df_E.shape

(11266, 4)

In [29]:
df = pd.concat([df_D, df_E], axis=0, ignore_index=True)
df.to_csv("tn_visit_area_info_total.csv", index=False)
df.shape

(22532, 4)

In [11]:
df['userID'] = pd.factorize(df['userID'])[0]
df['rating'] = df['rating'].replace('', 0).astype(float)

In [16]:
df.shape

(22532, 4)

In [12]:
train, test = python_chrono_split(df, 0.75)

In [13]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [14]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [15]:
train.shape

(17083, 4)

In [17]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-p

In [19]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 50
BATCH_SIZE = 128

SEED = 42

In [31]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="GMF",
    n_factors=4,
    layer_sizes=[16, 8, 4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=5e-4,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [36]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 619.5021 seconds for training.


In [37]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 19.9741 seconds for prediction.


In [38]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

/usr/local/lib/python3.10/dist-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/usr/local/lib/python3.10/dist-packages/recommenders/evaluation/python_evaluation.py:439: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  rating_true_common.groupby(col_user, as_index=False)[col_user].agg(
/usr/local/lib/python3.10/dist-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/usr/local/lib/python3.10/dist-packages/recomm

MAP:	0.021174
NDCG:	0.035714
Precision@K:	0.010777
Recall@K:	0.044587


/usr/local/lib/python3.10/dist-packages/recommenders/evaluation/python_evaluation.py:438: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
/usr/local/lib/python3.10/dist-packages/recommenders/evaluation/python_evaluation.py:439: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  rating_true_common.groupby(col_user, as_index=False)[col_user].agg(


## NCF Result
MAP:	0.026575 \\
NDCG:	0.049740 \\
Precision@10:	0.015909 \\
Recall@10:	0.066401

```
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16, 8, 4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=5e-4,
    verbose=10,
    seed=SEED
)
```



In [33]:
# Make a list of (user, item, rating) tuples
dataset = []
cornac_df = df[['userID', 'itemID', 'rating']]

for index, row in cornac_df.iterrows():
    dataset.append((row['userID'], row['itemID'], row['rating']))

dataset[:10]

[('a_a008632', 2210300003, 5.0),
 ('d_d004269', 2210220007, 2.0),
 ('c_c000751', 2209070004, 5.0),
 ('d_d000233', 2208200006, 4.0),
 ('d_d000480', 2208260005, 5.0),
 ('d_d011528', 2211060002, 5.0),
 ('d_d002165', 2210040001, 4.0),
 ('d_d009186', 2210270003, 5.0),
 ('d_d002138', 2210270002, 3.0),
 ('d_d012344', 2211130002, 3.0)]

In [34]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF
from cornac.metrics import RMSE, MAE, Precision, Recall

# 평가 방법 설정: 훈련 80%, 테스트 20%
ratio_split = RatioSplit(data=dataset, test_size=0.2, rating_threshold=4.0, verbose=True)

# Matrix Factorization (MF) 모델 생성
mf_model = MF(k=10, max_iter=50, learning_rate=0.01, lambda_reg=0.02, verbose=True)

# 성능 평가를 위한 실험 설정
exp = cornac.Experiment(eval_method=ratio_split, models=[mf_model], metrics=[RMSE(), MAE(), Precision(k=10), Recall(k=10)])

# 모델 학습 및 평가
exp.run()

rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 3487
Number of items = 1061
Number of ratings = 10805
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1
---
Test data:
Number of users = 3487
Number of items = 1061
Number of ratings = 3998
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 3487
Total items = 1061

[MF] Training started!


/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 7220 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 413 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!

[MF] Evaluation started!


Rating:   0%|          | 0/3998 [00:00<?, ?it/s]

Ranking:   0%|          | 0/2318 [00:00<?, ?it/s]


TEST:
...
   |    MAE |   RMSE | Precision@10 | Recall@10 | Train (s) | Test (s)
-- + ------ + ------ + ------------ + --------- + --------- + --------
MF | 0.5549 | 0.5842 |       0.0007 |    0.0045 |    0.6247 |   2.1707



In [35]:
import cornac

# Split data based on ratio
rs = cornac.eval_methods.RatioSplit(
    data=dataset, test_size=0.2, rating_threshold=4.0, seed=123
)

# Here we are comparing biased MF, PMF, and BPR
mf = cornac.models.MF(
    k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123
)
pmf = cornac.models.PMF(
    k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=123
)
bpr = cornac.models.BPR(
    k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123
)

most_pop = cornac.models.MostPop()

item_knn_bm25 = cornac.models.ItemKNN(
        k=100, similarity="cosine", weighting="bm25", name="ItemKNN-BM25"
    )
bpr = cornac.models.BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123)
wmf = cornac.models.WMF(
    k=5,
    max_iter=50,
    learning_rate=0.001,
    lambda_u=0.01,
    lambda_v=0.01,
    verbose=True,
    seed=123,
)
vaecf = cornac.models.VAECF(
    k=5,
    autoencoder_structure=[20],
    act_fn="tanh",
    likelihood="mult",
    n_epochs=100,
    batch_size=100,
    learning_rate=0.001,
    beta=1.0,
    seed=123,
    use_gpu=True,
    verbose=True,
)

# Define metrics used to evaluate the models
mae = cornac.metrics.MAE()
rmse = cornac.metrics.RMSE()
prec = cornac.metrics.Precision(k=10)
recall = cornac.metrics.Recall(k=10)
ndcg = cornac.metrics.NDCG(k=10)
auc = cornac.metrics.AUC()
mAP = cornac.metrics.MAP()

# Put it together into an experiment and run
cornac.Experiment(
    eval_method=rs,
    models=[mf, pmf, bpr, vaecf, item_knn_bm25],
    metrics=[mae, rmse, prec, recall, ndcg, auc, mAP],
    user_based=True,
    save_dir="save_models"
).run()

/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 7190 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 399 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


  0%|          | 0/100 [00:00<?, ?it/s]

VAECF model is saved to save_models/VAECF/2024-09-10_07-36-48-848174.pkl


  0%|          | 0/1064 [00:00<?, ?it/s]

ItemKNN-BM25 model is saved to save_models/ItemKNN-BM25/2024-09-10_07-36-55-923323.pkl

TEST:
...
             |    MAE |   RMSE |    AUC |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------ + ------ + ------ + ------ + ------ + ------- + ------------ + --------- + --------- + --------
MF           | 0.6175 | 0.6458 | 0.5487 | 0.0073 |  0.0034 |       0.0011 |    0.0069 |    0.0149 |   6.4014
PMF          | 0.5588 | 0.5918 | 0.9504 | 0.3309 |  0.3756 |       0.0723 |    0.5172 |    0.3913 |   3.6701
BPR          | 2.9805 | 3.0205 | 0.7814 | 0.0375 |  0.0382 |       0.0100 |    0.0732 |    0.3671 |   3.0501
VAECF        | 3.0624 | 3.1014 | 0.9682 | 0.1808 |  0.2278 |       0.0608 |    0.4223 |   20.4455 |   6.6946
ItemKNN-BM25 | 0.4683 | 0.4958 | 0.7262 | 0.0390 |  0.0675 |       0.0171 |    0.1304 |    0.1610 |   6.8944



In [ ]:
# MAP: 0.026575
# NDCG: 0.049740
# Precision@10: 0.015909
# Recall@10: 0.066401

## Hyperparameter Tuning in PMF using RandomSearchCV

In [47]:
import numpy as np
import cornac
from cornac.eval_methods import RatioSplit
from cornac.metrics import Recall, NDCG, Precision

# 하이퍼파라미터 범위 설정
param_grid = {
    'k': np.random.choice([5, 10, 20, 50]),  # latent factors 수
    'learning_rate': np.random.uniform(0.0001, 0.01, 5),  # 학습률
    'lambda_reg': np.random.uniform(0.0001, 0.01, 5)  # regularization strength
}

best_score = -np.inf
best_params = {}

# Split data based on ratio
rs = cornac.eval_methods.RatioSplit(
    data=dataset, test_size=0.2, rating_threshold=4.0, seed=123
)

# Random SearchCV 루프
for i in range(10):  # 10번의 랜덤 탐색
    params = {key: np.random.choice(values) for key, values in param_grid.items()}

    # PMF 모델 생성
    print(params)
    pmf = cornac.models.PMF(k=params['k'], max_iter=100, learning_rate=params['learning_rate'], lambda_reg=params['lambda_reg'])
    bpr = cornac.models.BPR(k=params['k'], max_iter=100, learning_rate=params['learning_rate'], lambda_reg=params['lambda_reg'])

    # 실험 실행
    experiment = cornac.Experiment(eval_method=rs, models=[pmf, bpr], metrics=[Precision(k=10), Recall(k=10), NDCG(k=10)]).run()

# print(f"Best Recall: {best_score}, Best Params: {best_params}")

/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 7190 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 399 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


{'k': 4, 'learning_rate': 0.0051753410266992894, 'lambda_reg': 0.003791852423253074}

TEST:
...
    | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------- + ------------ + --------- + --------- + --------
PMF |  0.0698 |       0.0189 |    0.1436 |    0.1750 |   1.0575
BPR |  0.0402 |       0.0102 |    0.0749 |    0.3771 |   2.6034

{'k': 10, 'learning_rate': 0.0051753410266992894, 'lambda_reg': 0.006582830759330975}

TEST:
...
    | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------- + ------------ + --------- + --------- + --------
PMF |  0.4162 |       0.0806 |    0.5742 |    0.3719 |   1.4378
BPR |  0.0369 |       0.0097 |    0.0711 |    0.1964 |   1.1087

{'k': 4, 'learning_rate': 0.0051753410266992894, 'lambda_reg': 0.003791852423253074}

TEST:
...
    | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
--- + ------- + ------------ + --------- + --------- + --------
PMF |  0.0709 |       0.0186 |    0.1433 |    0.1735 |   0.9186
BPR |

## 최적의 파라미터
{'k': 18, 'learning_rate': 0.0028446565782696115, 'lambda_reg': 0.009501331277498968}

\\

TEST:

\
    | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s) \\
--- + ------- + ------------ + --------- + --------- + -------- \\
PMF |  0.6364 |       0.1121 |    0.7655 |    0.8582 |   3.0917 \\
BPR |  0.0406 |       0.0102 |    0.0751 |    0.2136 |   1.1878 \\